In [1]:
%load_ext kedro.extras.extensions.ipython

[02/06/24 16:41:10] WARNING  /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/e ]8;id=591353;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=717991;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/warnings.py#109\109]8;;\
                             xtras/extensions/ipython.py:18: DeprecationWarning:                                   
                             kedro.extras.extensions.ipython should be accessed only using the                     
                             alias kedro.ipython. The unaliased name will be removed in Kedro                      
                             0.19.0.                                                                               
                               warnings.warn(                                                                      
                                                                                                                   

                    INFO     Resolved project path as: /home/sossa/raftel.                          ]8;id=910853;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=738319;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#134\134]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[02/06/24 16:41:10] INFO     Kedro project raftel                                                   ]8;id=688590;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=289907;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=733970;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=311295;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#104\104]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=742715;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=319916;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#110\110]8;;\

In [2]:
%reload_kedro

                    INFO     Resolved project path as: /home/sossa/raftel.                          ]8;id=595267;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=466748;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#134\134]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[02/06/24 16:41:10] INFO     Kedro project raftel                                                   ]8;id=27009;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=676427;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=991401;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=183897;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#104\104]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=885456;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=16142;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#110\110]8;;\

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import pyspark.sql.functions as F
from pyspark.sql.window import Window
import math

In [ ]:
master_table = catalog.load("master_table@spark")

### Create cyclical features for the time

In [ ]:
master_table = master_table.withColumn(
    "SEC2",
    F.col("SEC") - (F.col("DAY") - 1) *  86400
).withColumn(
    "SEC_SIN",
    F.sin(2 * math.pi * F.col("SEC2")/86400)
).withColumn(
    "SEC_COS",
    F.cos(2 * math.pi * F.col("SEC2")/86400)
)

### Create dummies for DAY

In [ ]:
expresiones_columnas_no_numericas = []
columna = "DAY"

unicos_columna = (
    master_table.select(columna)
    .distinct()
    .rdd.flatMap(lambda x: x)
    .collect()
)

unicos_columna.sort()

expr_unicos_columna = [
    F.when(
        F.col(columna)
        == valor_unico,
        1,
    )
    .otherwise(0)
    .alias(
        "{columna}_{valor}".format(
            columna=columna,
            valor=int(valor_unico),
        )
    )
    for valor_unico in unicos_columna
    if valor_unico != 1 #tiene un efecto parecido que drop_first=True
]

expresiones_columnas_no_numericas += expr_unicos_columna

In [ ]:
master_table = master_table.select("*", *expresiones_columnas_no_numericas)

In [ ]:
master_table = master_table.select(
    'SEC_SIN',
    'SEC_COS',
    'Ph1',
    'Ph2',
    'Ir1',
    'Fo1',
    'Fo2',
    'Di3',
    'Di4',
    'Ph3',
    'Ph4',
    'Ph5',
    'Ph6',
    'Co1',
    'Co2',
    'Co3',
    'So1',
    'So2',
    'Di1',
    'Di2',
    'Te1',
    'Fo3',
    'DAY_2',
    'DAY_3',
    'DAY_4',
    'DAY_5',
    'DAY_6',
    'DAY_7',
    'DAY_8',
    'DAY_9',
    'DAY_10',
    'DAY_11',
    'DAY_12',
    'DAY_13',
    'DAY_14',
    'DAY_15',
    'DAY_16',
    'DAY_17',
    'DAY_18',
    'DAY_19',
    'DAY_20',
    'DAY_21',
    'DAY_22',
    'DAY_23',
    'DAY_24',
    'DAY_25',
    'DAY_26',
    'DAY_27',
    'DAY_28',
    'DAY_29',
    'DAY_30',
    'Lag3_LABEL',
    'Lag2_LABEL',
    'Lag1_LABEL',
    'LABEL'
)

In [ ]:
catalog.save("featured_master_table@spark", master_table)

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split, KFold
import xgboost as xgb
from sklearn.metrics import accuracy_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

[02/06/24 16:41:18] WARNING  /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/future/ ]8;id=583440;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=679711;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/warnings.py#109\109]8;;\
                             standard_library/__init__.py:65: DeprecationWarning: the imp module is                
                             deprecated in favour of importlib; see the module's documentation for                 
                             alternative uses                                                                      
                               import imp                                                                          
                                                                                                                   

### Train and test split

In [5]:
X_aras = catalog.load("featured_master_table@pandas")

                    INFO     Loading data from 'featured_master_table@pandas'                   ]8;id=174790;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=225444;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             (ParquetDataSet)...                                                                   

In [6]:
y_aras = X_aras['LABEL']
X_aras = X_aras.drop(columns=['LABEL'])

In [7]:
X_aras_train, X_aras_test, y_aras_train, y_aras_test = train_test_split(X_aras, y_aras, test_size = 0.3, random_state = 0)

In [8]:
X_aras_train.shape, X_aras_test.shape, y_aras_train.shape, y_aras_test.shape

((1811203, 54), (776230, 54), (1811203,), (776230,))

## Experiments

### Cross Validation

### 1. Model with XGBoost and encoded activities in one label. No windowing.

In [9]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0,
        'tree_method':'hist', 
        'device': 'cuda:1',
        'predictor':'gpu_predictor',
        'n_jobs': -1
    }

In [10]:
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']), tree_method=space['tree_method'], device=space['device'],
                    n_jobs=space['n_jobs'], eval_metric="auc", early_stopping_rounds=10
    )
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation,
            verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [11]:
kf = KFold(n_splits=10)
best_hyperparams = []

for i, (train_index, test_index) in enumerate(kf.split(X_aras_train)):
    print(f"Fold {i}:")

    X_train = X_aras_train.iloc[train_index]
    y_train = y_aras_train.iloc[train_index]
    
    X_test = X_aras_train.iloc[test_index]
    y_test = y_aras_train.iloc[test_index]

    trials = Trials()
    best_hyperparams += [fmin(fn = objective,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 100,
                            trials = trials)]

Fold 0:
  0%|                                                                                                                             | 0/100 [00:00<?, ?trial/s, best loss=?]

[02/06/24 16:41:30] WARNING  /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/xgboost ]8;id=447543;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=732808;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/warnings.py#109\109]8;;\
                             /core.py:160: UserWarning: [16:41:30] WARNING:                                        
                             /workspace/src/context.cc:53: Only 1 GPUs are visible, setting device                 
                             ordinal to 0                                                                          
                               warnings.warn(smsg, UserWarning)                                                    
                                                                                                                   

[02/06/24 16:41:45] ERROR    job exception: [16:41:45]                                                  ]8;id=527227;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/hyperopt/fmin.py\fmin.py]8;;\:]8;id=215964;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/hyperopt/fmin.py#180\180]8;;\
                             /workspace/src/c_api/../data/../common/device_helpers.cuh:431: Memory                 
                             allocation error on worker 0: std::bad_alloc: cudaErrorMemoryAllocation:              
                             out of memory                                                                         
                             - Free memory: 0                                                                      
                             - Requested memory: 7224523424                                                        
                                                                                                                   
                             Stack trace:                                                                          
                               [bt] (0)                                                                            
                             /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/xgboost/lib            
                             /libxgboost.so(+0x77f79a) [0x7f23df05f79a]                                            
                               [bt] (1)                                                                            
                             /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/xgboost/lib            
                             /libxgboost.so(+0x783994) [0x7f23df063994]                                            
                               [bt] (2)                                                                            
                             /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/xgboost/lib            
                             /libxgboost.so(+0x91108b) [0x7f23df1f108b]                                            
                               [bt] (3)                                                                            
                             /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/xgboost/lib            
                             /libxgboost.so(+0x925118) [0x7f23df205118]                                            
                               [bt] (4)                                                                            
                             /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/xgboost/lib            
                             /libxgboost.so(+0x925175) [0x7f23df205175]                                            
                               [bt] (5)                                                                            
                             /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/xgboost/lib            
                             /libxgboost.so(+0x8abc07) [0x7f23df18bc07]                                            
                               [bt] (6)                                                                            
                             /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/xgboost/lib            
                             /libxgboost.so(+0x4f68fe) [0x7f23dedd68fe]                                            
                               [bt] (7)                                                                            
                             /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/xgboost/lib            
                             /libxgboost.so(+0x4efea5) [0x7f23dedcfea5]                                            
                               [bt] (8)                           

  0%|                                                                                                                             | 0/100 [00:16<?, ?trial/s, best loss=?]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:14                                                                                   │
│                                                                                                  │
│   11 │   y_test = y_aras_train.iloc[test_index]                                                  │
│   12 │                                                                                           │
│   13 │   trials = Trials()                                                                       │
│ ❱ 14 │   best_hyperparams += [fmin(fn = objective,                                               │
│   15 │   │   │   │   │   │   │   space = space,                                                  │
│   16 │   │   │   │   │   │   │   algo = tpe.suggest,                                             │
│   17 │   │   │   │   │   │   │   max_evals = 100,                                                │
│                                                                                                  │
│ /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/hyperopt/fmin.py:540 in fmin      │
│                                                                                                  │
│   537 │   │   fn = __objective_fmin_wrapper(fn)                                                  │
│   538 │                                                                                          │
│   539 │   if allow_trials_fmin and hasattr(trials, "fmin"):                                      │
│ ❱ 540 │   │   return trials.fmin(                                                                │
│   541 │   │   │   fn,                                                                            │
│   542 │   │   │   space,                                                                         │
│   543 │   │   │   algo=algo,                                                                     │
│                                                                                                  │
│ /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/hyperopt/base.py:671 in fmin      │
│                                                                                                  │
│    668 │   │   #    but for now it's still sitting in another file.                              │
│    669 │   │   from .fmin import fmin                                                            │
│    670 │   │                                                                                     │
│ ❱  671 │   │   return fmin(                                                                      │
│    672 │   │   │   fn,                                                                           │
│    673 │   │   │   space,                                                                        │
│    674 │   │   │   algo=algo,                                                                    │
│                                                                                                  │
│ /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/hyperopt/fmin.py:586 in fmin      │
│                                                                                                  │
│   583 │   rval.catch_eval_exceptions = catch_eval_exceptions                                     │
│   584 │                                                                                          │
│   585 │   # next line is where the fmin is actually executed                                     │
│ ❱ 586 │   rval.exhaust()                                                                         │
│   587 │                                                                                          │
│   588 │   if return_argmin:                                                                      │
│   589 │   │   if len(trials.trials) == 0:                  

In [ ]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

### Not used ↓↓↓↓

In [ ]:
X_aras_train = X_aras.loc[(
    (X_aras['DAY_26'] != 1) & 
    (X_aras['DAY_27'] != 1) & 
    (X_aras['DAY_28'] != 1) & 
    (X_aras['DAY_29'] != 1) &
    (X_aras['DAY_30'] != 1) 
)]

X_aras_test = X_aras.loc[(
    (X_aras['DAY_26'] == 1) | 
    (X_aras['DAY_27'] == 1) | 
    (X_aras['DAY_28'] == 1) | 
    (X_aras['DAY_29'] == 1) | 
    (X_aras['DAY_30'] == 1) 
)]

In [ ]:
y_aras_train = X_aras_train['LABEL']
y_aras_test = X_aras_test['LABEL']


X_aras_train = X_aras_train.drop(columns=['LABEL'])
X_aras_test = X_aras_test.drop(columns=['LABEL'])

In [ ]:
X_aras_train.shape, y_aras_train.shape, X_aras_test.shape, y_aras_test.shape